# Pitfalls: Common ways to introduce bugs in your Python code

## Part of the Python Best Practices course, https://ssciwr.github.io/Python-best-practices-course/
### February 2026, I. S. Ulusoy 

## Naming of the module

A source of errors can be naming a module the same as another module that is imported, in this example the module is named `math.py` but also imports math from the standard Python library; and function calls using methods from the math module will fail, as Python will look for those in the `math.py` file. 

In [ ]:
!python math.py

**Solution**: Name your module file different than the modules that you are importing.

## Shallow and deep copies

When copying lists and other mutable variable types like dictionarys, using an `=` sign only points the new variable to the same position in memory as the initial one. Changing one then automatically changes the other.

In [ ]:
import copy

In [ ]:
mylist1 = [1, 2, 3]
mylist2 = mylist1
print("list1 is", mylist1)
print("list2 is", mylist2)

In [ ]:
mylist2.append(4)
print("list1 is", mylist1)

In [ ]:
print("list2 is", mylist2)

In [ ]:
# notice that both point to the same physical address
print("id of list1:", id(mylist1))
print("id of list2:", id(mylist2))

**Solution**: Use `copy` or `deepcopy` instead to create copies of objects.

### Shallow copy

In [ ]:
mylist3 = mylist1.copy()
print("list3 is", mylist3)

In [ ]:
mylist3.append("x")
print("list1 is", mylist1)
print("list3 is", mylist3)

In [ ]:
mylist3[3] = "r"
print("list1 is", mylist1)
print("list3 is", mylist3)

### Deep copy

Consider nesting:

In [ ]:
deeplist1 = [[1, 2], [3, 4]]
deeplist2 = deeplist1.copy()
print("deeplist1 is", deeplist1)
print("deeplist2 is", deeplist2)

In [ ]:
deeplist1[1][0] = "x"
print("deeplist1 is", deeplist1)
print("deeplist2 is", deeplist2)

In [ ]:
# notice that at the nested level both lists have changed
# this is because they are referencing the same physical addresses for the list items:
for item, item2 in zip(deeplist1, deeplist2):
    print(id(item))
    print(id(item2))

In [ ]:
# so for nested lists you need a deep copy:
deeplist3 = copy.deepcopy(deeplist1)
deeplist1[1][1] = "y"
print("deeplist1 is", deeplist1)
print("deeplist3 is", deeplist3)

In [ ]:
for item, item2 in zip(deeplist1, deeplist3):
    print(id(item))
    print(id(item2))

## Instantiation of mutable default keyword arguments in function calls
Default arguments are only evaluated once: At the time the function is created. If you provide a mutable default keyword argument and then change it in the function, the next time the function is called without that keyword, the default will point to the same address as in the first call; but the argument will have already changed, so the default in the first call and the default in the second call are different. 

In [ ]:
def ingredients(ingredient, all_ingredients=[]):
    all_ingredients.append(ingredient)
    return all_ingredients


# here, all_ingredients is a list, so mutable
# but it is instantiated as an empty list

In [ ]:
print(ingredients.__defaults__)

In [ ]:
all_ingredients = ingredients("flour")
print(all_ingredients)

In [ ]:
all_ingredients = ingredients("sugar")
print(all_ingredients)

In [ ]:
all_ingredients = ingredients("butter")
print(all_ingredients)

In [ ]:
print(ingredients.__defaults__)

Since the default argument is mutable, it is shared across all calls to the function. So when we append to it in one call, it changes for all calls that use the default argument.

**Solution**: Only provide non-mutable default arguments.

In [ ]:
def ingredients(ingredient, all_ingredients=None):
    if all_ingredients is None:
        all_ingredients = []
    all_ingredients.append(ingredient)
    return all_ingredients


# None is not mutable and is instantiated as None, so it is not shared across function calls

In [ ]:
print(ingredients.__defaults__)

In [ ]:
all_ingredients = ingredients("flour")
print(all_ingredients)

In [ ]:
print(ingredients.__defaults__)

In [ ]:
all_ingredients = ingredients("butter")
print(all_ingredients)

In [ ]:
all_ingredients = ingredients("flour")
all_ingredients = ingredients("butter", all_ingredients)
print(all_ingredients)

Now we get the desired behaviour. This also applies to dictionaries and objects:

In [ ]:
def myfunc(a={"b": 0}):
    a["b"] += 5
    print(a)

In [ ]:
myfunc()

In [ ]:
myfunc()

In [ ]:
myfunc()

In [ ]:
# again instantiate the dict as None
def myfunc(a=None):
    if a is None:
        a = {"b": 0}
    a["b"] += 5
    print(a)

In [ ]:
myfunc()

In [ ]:
myfunc()

In [ ]:
myfunc({"b": 2})

In [ ]:
# for objects
from datetime import datetime


def display_time(time_to_print=datetime.now()):
    print(time_to_print.strftime("%B %d, %Y %H:%M:%S"))

In [ ]:
display_time()

In [ ]:
display_time()

In [ ]:
display_time()

In [ ]:
# evaluate the function explictly
def display_time(time_to_print=None):
    if time_to_print is None:
        time_to_print = datetime.now()
    print(time_to_print.strftime("%B %d, %Y %H:%M:%S"))

In [ ]:
display_time()

In [ ]:
display_time()

## Exhausting iterators

Iterators and generators can be exhausted, meaning you can only use them once. 

In [ ]:
def exhaust_my_iterators(days, lunch):
    menu = zip(days, lunch)
    print("Printing the list(menu):", list(menu))
    full_menu = []
    print("Printing the menu item by item:")
    for item in menu:
        print("On {} we offer {} for lunch.".format(item[0], item[1]))
        full_menu.append((item[0], item[1]))
    return full_menu

In [ ]:
days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]
lunch = ["Pizza", "Salad", "Pasta", "Sushi", "Sandwich"]
menu = exhaust_my_iterators(days, lunch)

In [ ]:
print(menu)

In [ ]:
def exhaust_my_iterators(days, lunch):
    menu = zip(days, lunch)
    # do not access the iterator before the loop
    full_menu = []
    print("Printing the menu item by item:")
    for item in menu:  # now the "menu" is not exhausted before the loop
        print("On {} we offer {} for lunch.".format(item[0], item[1]))
        full_menu.append((item[0], item[1]))
    return full_menu

In [ ]:
days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]
lunch = ["Pizza", "Salad", "Pasta", "Sushi", "Sandwich"]
menu = exhaust_my_iterators(days, lunch)

In [ ]:
# you can also explicitly convert the iterator to a list
# not recommended for large iterators
def exhaust_my_iterators(days, lunch):
    menu = list(zip(days, lunch))
    print("Printing the list(menu):", list(menu))
    full_menu = []
    print("Printing the menu item by item:")
    for item in menu:  # now the "menu" is not exhausted before the loop
        print("On {} we offer {} for lunch.".format(item[0], item[1]))
        full_menu.append((item[0], item[1]))
    return full_menu

In [ ]:
days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]
lunch = ["Pizza", "Salad", "Pasta", "Sushi", "Sandwich"]
menu = exhaust_my_iterators(days, lunch)

**Solution**: If you create an iterator or a generator and you need it more than once you need to save it first. As in the example provided, the iterator is created using `zip`, and can be saved in a `list`. Better is to avoid accessing it multiple times.

## Variable assignment in different scopes

Assigning a variable within a function shadows any assignment that may have happened in an outer scope. 


In [ ]:
# used for testing, please do not modify
def testing():
    myfunc()
    print(mylist, "value of variable in outer scope")

In [ ]:
# global variable
mylist = [1, 2, 3]

In [ ]:
def myfunc():
    # local variable
    mylist = ["x"]
    # mylist is now shadowed
    print(mylist, "value of variable in the inner scope")

In [ ]:
myfunc()
testing()

**Solution**: Pass the variable as an argument into the inner scope or use the return value of a new assignment.